# Save to S3 with a SageMaker Processing Job

<div class="alert alert-info"> 💡 <strong> Quick Start </strong>
To save your processed data to S3, select the Run menu above and click <strong>Run all cells</strong>. 
<strong><a style="color: #0397a7 " href="#Job-Status-&-S3-Output-Location">
    <u>View the status of the export job and the output S3 location</u></a>.
</strong>
</div>


This notebook executes your Data Wrangler Flow `Basketball.flow` on the entire dataset using a SageMaker 
Processing Job and will save the processed data to S3.

This notebook saves data from the step `Encode Categorical` from `Source: Dataset_2200_Baskball.Csv`. To save from a different step, go to Data Wrangler 
to select a new step to export. 

---

## Contents

1. [Inputs and Outputs](#Inputs-and-Outputs)
1. [Run Processing Job](#Run-Processing-Job)
   1. [Job Configurations](#Job-Configurations)
   1. [Create Processing Job](#Create-Processing-Job)
   1. [Job Status & S3 Output Location](#Job-Status-&-S3-Output-Location)
1. [Optional Next Steps](#(Optional)Next-Steps)
    1. [Load Processed Data into Pandas](#(Optional)-Load-Processed-Data-into-Pandas)
    1. [Train a model with SageMaker](#(Optional)Train-a-model-with-SageMaker)
---

# Inputs and Outputs

The below settings configure the inputs and outputs for the flow export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

In <b>Input - Source</b> you can configure the data sources that will be used as input by Data Wrangler

1. For S3 sources, configure the source attribute that points to the input S3 prefixes
2. For all other sources, configure attributes like query_string, database in the source's 
<b>DatasetDefinition</b> object.

If you modify the inputs the provided data must have the same schema and format as the data used in the Flow. 
You should also re-execute the cells in this section if you have modified the settings in any data sources.

Parametrized data sources will be ignored when creating ProcessingInputs, and will directly read from the source.
Network isolation is not supported for parametrized data sources.
</div>

In [2]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

data_sources = []

## Input - S3 Source: dataset_2200_baskball.csv

In [3]:
data_sources.append(ProcessingInput(
    source="s3://baskball-bucket/dataset_2200_baskball.csv", # You can override this to point to other dataset on S3
    destination="/opt/ml/processing/dataset_2200_baskball.csv",
    input_name="dataset_2200_baskball.csv",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
))

## Output: S3 settings

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. <b>bucket</b>: you can configure the S3 bucket where Data Wrangler will save the output. The default bucket from 
the SageMaker notebook session is used. 
2. <b>flow_export_id</b>: A randomly generated export id. The export id must be unique to ensure the results do not 
conflict with other flow exports 
3. <b>s3_ouput_prefix</b>:  you can configure the directory name in your bucket where your data will be saved.
</div>

In [4]:
import time
import uuid
import boto3
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

region = boto3.Session().region_name

# You can configure this with your own bucket name, e.g.
# bucket = "my-bucket"
bucket = sess.default_bucket()
print(f"Data Wrangler export storage bucket: {bucket}")

# unique flow export ID
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

Data Wrangler export storage bucket: sagemaker-ca-central-1-224670572127


Below are the inputs required by the SageMaker Python SDK to launch a processing job.

In [5]:
# Output name is auto-generated from the select node's ID + output name from the flow file.
output_name = "1252db56-2cd1-415b-8846-ced1562b5e9e.default"

s3_output_prefix = f"export-{flow_export_name}/output"
s3_output_base_path = f"s3://{bucket}/{s3_output_prefix}"
print(f"Processing output base path: {s3_output_base_path}\nThe final output location will contain additional subdirectories.")

processing_job_output = ProcessingOutput(
    output_name=output_name,
    source="/opt/ml/processing/output",
    destination=s3_output_base_path,
    s3_upload_mode="EndOfJob"
)

Processing output base path: s3://sagemaker-ca-central-1-224670572127/export-flow-08-14-47-04-528a569e/output
The final output location will contain additional subdirectories.


## Upload Flow to S3

To use the Data Wrangler as an input to the processing job,  first upload your flow file to Amazon S3.

In [6]:
import os
import json
import boto3

# name of the flow file which should exist in the current notebook working directory
flow_file_name = "Basketball.flow"

# Load .flow file from current notebook working directory 
!echo "Loading flow file from current notebook working directory: $PWD"

with open(flow_file_name) as f:
    flow = json.load(f)

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"data_wrangler_flows/{flow_export_name}.flow", ExtraArgs={"ServerSideEncryption": "aws:kms"})

flow_s3_uri = f"s3://{bucket}/data_wrangler_flows/{flow_export_name}.flow"

print(f"Data Wrangler flow {flow_file_name} uploaded to {flow_s3_uri}")

Loading flow file from current notebook working directory: /root
Data Wrangler flow Basketball.flow uploaded to s3://sagemaker-ca-central-1-224670572127/data_wrangler_flows/flow-08-14-47-04-528a569e.flow


The Data Wrangler Flow is also provided to the Processing Job as an input source which we configure below.

In [7]:
## Input - Flow: Basketball.flow
flow_input = ProcessingInput(
    source=flow_s3_uri,
    destination="/opt/ml/processing/flow",
    input_name="flow",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
)

# Run Processing Job 
## Job Configurations

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

You can configure the following settings for Processing Jobs. If you change any configurations you will 
need to re-execute this and all cells below it by selecting the Run menu above and click 
<b>Run Selected Cells and All Below</b>

1. IAM role for executing the processing job. 
2. A unique name of the processing job. Give a unique name every time you re-execute processing jobs
3. Data Wrangler Container URL.
4. Instance count, instance type and storage volume size in GB.
5. Content type for each output. Data Wrangler supports CSV as default and Parquet.
6. Network Isolation settings
7. KMS key to encrypt output data
</div>

In [8]:
from sagemaker import image_uris

# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# Unique processing job name. Give a unique name every time you re-execute processing jobs.
processing_job_name = f"data-wrangler-flow-processing-{flow_export_id}"

# Data Wrangler Container URL.
container_uri = "557239378090.dkr.ecr.ca-central-1.amazonaws.com/sagemaker-data-wrangler-container:2.x"
# Pinned Data Wrangler Container URL.
container_uri_pinned = "557239378090.dkr.ecr.ca-central-1.amazonaws.com/sagemaker-data-wrangler-container:2.3.0"

# Processing Job Instance count and instance type.
instance_count = 2
instance_type = "ml.m5.4xlarge"

# Size in GB of the EBS volume to use for storing data during processing.
volume_size_in_gb = 30


# Content type for each output. Data Wrangler supports CSV as default and Parquet.
output_content_type = "CSV"

# Delimiter to use for the output if the output content type is CSV. Uncomment to set.
# delimiter = ","

# Compression to use for the output. Uncomment to set.
# compression = "gzip"

# Configuration for partitioning the output. Uncomment to set.
# "num_partition" sets the number of partitions/files written in the output.
# "partition_by" sets the column names to partition the output by.
# partition_config = {
#     "num_partitions": 1,
#     "partition_by": ["column_name_1", "column_name_2"],
# }

# Network Isolation mode; default is off.
enable_network_isolation = False

# List of tags to be passed to the processing job.
user_tags = []

# Output configuration used as processing job container arguments. Only applies when writing to S3.
# Uncomment to set additional configurations.
output_config = {
    output_name: {
        "content_type": output_content_type,
        # "delimiter": delimiter,
        # "compression": compression,
        # "partition_config": partition_config,
    }
}

# Refit configuration determines whether Data Wrangler refits the trainable parameters on the entire dataset. 
# When True, the processing job relearns the parameters and outputs a new flow file.
# You can specify the name of the output flow file under 'output_flow'.
# Note: There are length constraints on the container arguments (max 256 characters).
refit_trained_params = {
    "refit": False,
    "output_flow": f"data-wrangler-flow-processing-{flow_export_id}.flow"
}

# KMS key for per object encryption; default is None.
kms_key = None

### (Optional) Configure Spark Cluster Driver Memory

In [9]:
# The Spark memory configuration. Change to specify the driver and executor memory in MB for the Spark cluster during processing.
driver_memory_in_mb = 2048
executor_memory_in_mb = 55742

config = json.dumps({
    "Classification": "spark-defaults",
    "Properties": {
        "spark.driver.memory": f"{driver_memory_in_mb}m",
        "spark.executor.memory": f"{executor_memory_in_mb}m"
    }
})

config_file = f"config-{flow_export_id}.json"
with open(config_file, "w") as f:
    f.write(config)

config_s3_path = f"spark_configuration/{processing_job_name}/configuration.json"
config_s3_uri = f"s3://{bucket}/{config_s3_path}"
s3_client.upload_file(config_file, bucket, config_s3_path, ExtraArgs={"ServerSideEncryption": "aws:kms"})
print(f"Spark Config file uploaded to {config_s3_uri}")
os.remove(config_file)

# Provides the spark config file to processing job and set the cluster driver memory. Uncomment to set.
# data_sources.append(ProcessingInput(
#     source=config_s3_uri,
#     destination="/opt/ml/processing/input/conf",
#     input_name="spark-config",
#     s3_data_type="S3Prefix",
#     s3_input_mode="File",
#     s3_data_distribution_type="FullyReplicated"
# ))

Spark Config file uploaded to s3://sagemaker-ca-central-1-224670572127/spark_configuration/data-wrangler-flow-processing-08-14-47-04-528a569e/configuration.json


## Create Processing Job

To launch a Processing Job, you will use the SageMaker Python SDK to create a Processor function.

In [10]:
# Setup processing job network configuration
from sagemaker.network import NetworkConfig

network_config = NetworkConfig(
        enable_network_isolation=enable_network_isolation,
        security_group_ids=['sg-0654aecb0fbe75fcc'],
        subnets=['subnet-0b68f9143765ebb5e', 'subnet-0061d0e1276cb46a6']
    )

In [11]:
from sagemaker.processing import Processor

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=instance_count,
    instance_type=instance_type,
    volume_size_in_gb=volume_size_in_gb,
    network_config=network_config,
    sagemaker_session=sess,
    output_kms_key=kms_key,
    tags=user_tags
)

# Start Job
processor.run(
    inputs=[flow_input] + data_sources, 
    outputs=[processing_job_output],
    arguments=[f"--output-config '{json.dumps(output_config)}'"] + [f"--refit-trained-params '{json.dumps(refit_trained_params)}'"],
    wait=False,
    logs=False,
    job_name=processing_job_name
)

INFO:sagemaker:Creating processing-job with name data-wrangler-flow-processing-08-14-47-04-528a569e


## Job Status & S3 Output Location

Below you wait for processing job to finish. If it finishes successfully, the raw parameters used by the 
Processing Job will be printed.

To prevent data of different processing jobs and different output nodes from being overwritten or combined, 
Data Wrangler uses the name of the processing job and the name of the output to write the output.

In [12]:
s3_job_results_path = f"{s3_output_base_path}/{processing_job_name}/{output_name.replace('.', '/')}"
print(f"Job results are saved to S3 path: {s3_job_results_path}")

job_result = sess.wait_for_processing_job(processing_job_name)
job_result

Job results are saved to S3 path: s3://sagemaker-ca-central-1-224670572127/export-flow-08-14-47-04-528a569e/output/data-wrangler-flow-processing-08-14-47-04-528a569e/1252db56-2cd1-415b-8846-ced1562b5e9e/default
...............................................................................!

{'ProcessingInputs': [{'InputName': 'flow',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ca-central-1-224670572127/data_wrangler_flows/flow-08-14-47-04-528a569e.flow',
    'LocalPath': '/opt/ml/processing/flow',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'dataset_2200_baskball.csv',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://baskball-bucket/dataset_2200_baskball.csv',
    'LocalPath': '/opt/ml/processing/dataset_2200_baskball.csv',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': '1252db56-2cd1-415b-8846-ced1562b5e9e.default',
    'S3Output': {'S3Uri': 's3://sagemaker-ca-central-1-224670572127/export-flow-08-14-47-04-528a569e/output',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode

## (Optional)Next Steps

Now that data is available on S3 you can use other SageMaker components that take S3 URIs as input such as 
SageMaker Training, Built-in Algorithms, etc. Similarly you can load the dataset into a Pandas dataframe 
in this notebook for further inspection and work. The examples below show how to do both of these steps.

By default optional steps do not run automatically, set `run_optional_steps` to True if you want to 
execute optional steps

In [13]:
run_optional_steps = False

In [14]:
# This will stop the below cells from executing if "Run All Cells" was used on the notebook.
if not run_optional_steps:
    raise SystemExit("Stop here. Do not automatically execute optional steps.")

SystemExit: Stop here. Do not automatically execute optional steps.

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### (Optional) Load Processed Data into Pandas

We use the [AWS SDK for pandas library](https://github.com/awslabs/aws-sdk-pandas) to load the exported 
dataset into a Pandas data frame for a preview of first 10000 rows.

To turn on automated visualizations and data insights for your pandas data frame, import the sagemaker_datawrangler library.

In [15]:
!pip install -q awswrangler pandas
import awswrangler as wr

# Import sagemaker_datawrangler to show visualizations and automated data
# quality insights, and export code to prepare data in a pandas data frame.
try:
    import sagemaker_datawrangler
except ImportError:
    print("sagemaker_datawrangler is not imported. Change your kernel to the Data Science 3.0 Kernel Image and try again.")
    pass

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.4 which is incompatible.

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [16]:
chunksize = 10000

if output_content_type.upper() == "CSV":
    dfs = wr.s3.read_csv(s3_job_results_path, chunksize=chunksize)
elif output_content_type.upper() == "PARQUET":
    dfs = wr.s3.read_parquet(s3_job_results_path, chunked=chunksize)
else:
    print(f"Unexpected output content type {output_content_type}") 

df = next(dfs)
df

INFO:root:{"event_type": "ganymede.initialization", "event_status": "start", "app_context": {"ganymede_version": "0.3.9", "app_metadata": {"AppType": "KernelGateway", "DomainId": "d-vm1emmiwg70d", "UserProfileName": "default-1678239153575", "ResourceArn": "arn:aws:sagemaker:ca-central-1:224670572127:app/d-vm1emmiwg70d/default-1678239153575/KernelGateway/sagemaker-data-scienc-ml-t3-medium-8f0c29904c6871ec44bae2489cde", "ResourceName": "sagemaker-data-scienc-ml-t3-medium-8f0c29904c6871ec44bae2489cde", "AppImageVersion": "", "Region": "ca-central-1", "AccountId": "224670572127"}}}
INFO:root:DataFrame size: row_count = 96, column_count = 7.
INFO:root:Computing on the top 10000 rows of the DataFrame.
INFO:root:Toggled to the sagemaker_datawrangler view.


    assists_per_minute  height  time_played   age  points_per_minute  \
0               0.0888   201.0        36.02  28.0             0.5885   
1               0.1399   198.0        39.32  30.0             0.8291   
2               0.0747   198.0        38.80  26.0             0.4974   
3               0.0983   191.0        40.71  30.0             0.5772   
4               0.1276   196.0        38.40  28.0             0.5703   
..                 ...     ...          ...   ...                ...   
91              0.1554   183.0        11.58  24.0             0.3195   
92              0.2282   185.0        10.08  24.0             0.2381   
93              0.1778   185.0        18.56  23.0             0.2802   
94              0.1863   185.0        11.81  23.0             0.3810   
95              0.1014   193.0        13.81  32.0             0.1593   

    sex_Female  sex_Male  
0          0.0       1.0  
1          0.0       1.0  
2          1.0       0.0  
3          1.0       0.0  


INFO:root:{"event_type": "ganymede.initialization", "event_status": "success", "app_context": {"ganymede_version": "0.3.9", "app_metadata": {"AppType": "KernelGateway", "DomainId": "d-vm1emmiwg70d", "UserProfileName": "default-1678239153575", "ResourceArn": "arn:aws:sagemaker:ca-central-1:224670572127:app/d-vm1emmiwg70d/default-1678239153575/KernelGateway/sagemaker-data-scienc-ml-t3-medium-8f0c29904c6871ec44bae2489cde", "ResourceName": "sagemaker-data-scienc-ml-t3-medium-8f0c29904c6871ec44bae2489cde", "AppImageVersion": "", "Region": "ca-central-1", "AccountId": "224670572127"}}, "metadata": {"latency": 0.09447813034057617}}


## (Optional)Train a model with SageMaker
Now that the data has been processed, you may want to train a model using the data. The following shows an 
example of doing so using a popular algorithm - XGBoost. For more information on algorithms available in 
SageMaker, see [Getting Started with SageMaker Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html). 
It is important to note that the following XGBoost objective ['binary', 'regression', 'multiclass'] 
hyperparameters, or content_type may not be suitable for the output data, and will require changes to 
train a proper model. Furthermore, for CSV training, the algorithm assumes that the target 
variable is in the first column. For more information on SageMaker XGBoost, 
see https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html.


### Set Training Data path
We set the training input data path from the output of the Data Wrangler processing job..

In [17]:
s3_training_input_path = s3_job_results_path
print(f"training input data path: {s3_training_input_path}")

training input data path: s3://sagemaker-ca-central-1-224670572127/export-flow-08-14-47-04-528a569e/output/data-wrangler-flow-processing-08-14-47-04-528a569e/1252db56-2cd1-415b-8846-ced1562b5e9e/default


### Configure the algorithm and training job

The Training Job hyperparameters are set. For more information on XGBoost Hyperparameters, 
see https://xgboost.readthedocs.io/en/latest/parameter.html.

In [18]:
region = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
hyperparameters = {
    "max_depth":"5",
    "objective": "reg:squarederror",
    "num_round": "10",
}
train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_training_input_path,
    content_type=train_content_type,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Start the Training Job

The TrainingJob configurations are set using the SageMaker Python SDK Estimator, and which is fit using 
the training data from the Processing Job that was run earlier.

In [19]:
estimator = sagemaker.estimator.Estimator(
    container,
    iam_role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
)
estimator.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-03-08-14-55-02-333


2023-03-08 14:55:02 Starting - Starting the training job...
2023-03-08 14:55:18 Starting - Preparing the instances for training...
2023-03-08 14:56:00 Downloading - Downloading input data...
2023-03-08 14:56:30 Training - Downloading the training image...
2023-03-08 14:56:45 Training - Training image download completed. Training in progress...[2023-03-08 14:57:04.742 ip-10-0-83-48.ca-central-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[2023-03-08 14:57:04.801 ip-1

Now that you have a trained model there are a number of different things you can do. 
For more details on training with SageMaker, please see 
https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html.